In [1]:
import pandas as pd
import numpy as np

In [2]:
full_data = pd.read_csv('./data/final_data/full_data.csv')

In [3]:
print(full_data.shape)

(6202174, 11)


In [4]:
districts = ["강서구","구로구","양천구","영등포구", "동작구", "금천구", "관악구", "서초구", "강남구", "송파구", "강동구", "마포구", "용산구", "성동구", "광진구", "중구", "서대문구", "은평구", "종로구", "성북구", "동대문구", "중랑구", "노원구", "도봉구", "강북구"]
full_data = full_data[(full_data['출발지구'].isin(districts)) & (full_data['목적지구'].isin(districts))]

In [5]:
print(len(full_data.출발지동.unique()))
print(full_data.출발지동.unique())

432
['명동' '반포1동' '인수동' '청림동' '월계1동' '청파동' '구로제3동' '방화제1동' '여의동' '잠실7동' '묵제2동'
 '잠실6동' '대치2동' '제기제2동' '월곡제2동' '수궁동' '월계3동' '서교동' '성수2가제1동' '회현동' '전농제2동'
 '교남동' '방학제1동' '효창동' '광희동' '남가좌제1동' '천호제2동' '구로제2동' '이태원제1동' '능동' '하계1동'
 '상봉제2동' '풍납2동' '등촌제3동' '합정동' '상계6.7동' '갈현제1동' '신당제3동' '거여1동' '방배2동'
 '양평제2동' '홍은제1동' '역촌동' '홍제제1동' '가양제2동' '이문제1동' '망우제3동' '종암동' '창제5동'
 '시흥제2동' '장지동' '신당제4동' '강일동' '화곡제6동' '개포1동' '면목제2동' '서초2동' '송중동' '신내1동'
 '중계2.3동' '면목제3.8동' '신내2동' '독산제4동' '고척제2동' '반포2동' '돈암제1동' '번제2동' '가산동'
 '성산제2동' '공항동' '암사제1동' '삼양동' '성수2가제3동' '휘경제2동' '성내제3동' '신대방제1동' '중계4동'
 '번제3동' '도화동' '가양제3동' '양재1동' '구로제5동' '잠원동' '대조동' '신길제4동' '번제1동' '구로제4동'
 '월계2동' '부암동' '증산동' '이촌제2동' '평창동' '공릉2동' '쌍문제4동' '도봉제2동' '녹번동' '공릉1.3동'
 '수유제2동' '상도제3동' '행운동' '길음제1동' '수유제1동' '명일제2동' '독산제2동' '우이동' '쌍문제3동' '은천동'
 '당산제1동' '삼선동' '정릉제4동' '상봉제1동' '창제1동' '금호2.3가동' '방학제3동' '서원동' '황학동' '송파1동'
 '중곡제4동' '삼성1동' '창제4동' '보문동' '상계3.4동' '수서동' '오류제2동' '후암동' '상일동' '신정3동'
 '마천2동' '중곡제2동' '둔촌제2동' '성현동' '독산제1동' '방화제3동' '응암제3동' '장안제4

In [6]:
# 영종동 drop
full_data.drop(full_data[full_data.출발지동 == "영종동"].index, inplace=True)

### 행정동별 주민 센터 위치 및 가장 가까운 차고지와의 거리 계산

In [7]:
community_center = pd.read_excel('./data/extra_data/행정동별주민센터.xlsx', sheet_name = 'seoul_edit')
taxi_garage = pd.read_excel('./data/extra_data/장애인콜택시차고지.xlsx')

In [8]:
# 행정동 공백 제거
community_center.행정동 = community_center.행정동.str.strip()

In [9]:
# 가장 가까운 차고지와의 거리 계산
def cal_min_distance(location):
    return np.sqrt(np.min(np.sum((taxi_garage.iloc[:,5:] - location) ** 2, axis = 1)))

community_center['nearest_garage_distance'] = community_center.iloc[:, 1:].apply(cal_min_distance, axis = 1)

In [10]:
community_center.head()

,행정동,Latitude,Longitude,nearest_garage_distance
0,청운효자동,37.584088,126.970609,0.011351
1,사직동,37.576172,126.968804,0.007382
2,삼청동,37.584949,126.981746,0.012402
3,부암동,37.592393,126.964026,0.021851
4,평창동,37.606363,126.968335,0.033199


In [11]:
call_taxi_dongs = set(full_data.출발지동.unique())
community_dongs = set(community_center.행정동.unique())
print(set.difference(call_taxi_dongs, community_dongs))
print(set.difference(community_dongs, call_taxi_dongs))

set()
{'동화동', '위례동', '청구동', '제기제3동', '약수동', '항동', '답십리제5동', '다산동'}


In [66]:
# join
final_data = pd.merge(full_data, community_center, how = "left", left_on = "출발지동", right_on = "행정동")
final_data.rename(columns = {"Latitude":"출발동_위도", "Longitude":"출발동_경도", "nearest_garage_distance":"출발동_최근접_차고지_거리"}, inplace = True)
final_data.drop(columns = ["행정동"], inplace = True)

In [67]:
final_data = pd.merge(final_data, community_center, how = "left", left_on = "목적지동", right_on = "행정동")
final_data.rename(columns = {"Latitude":"목적동_위도", "Longitude":"목적동_경도"}, inplace = True)
final_data.drop(columns = ["행정동", "nearest_garage_distance"], inplace = True)

In [68]:
final_data['이동거리'] = np.sqrt((final_data.출발동_위도 - final_data.목적동_위도) ** 2 + (final_data.출발동_경도 - final_data.목적동_경도) ** 2)

In [69]:
print(final_data.shape)
final_data.head()

(5817767, 17)


,dispatch_waiting_time,total_waiting_time,cancel_time,using_time,using_day,year,출발지구,출발지동,목적지구,목적지동,이용목적,출발동_위도,출발동_경도,출발동_최근접_차고지_거리,목적동_위도,목적동_경도,이동거리
0,136.0,165.0,NaN,새벽,평일,2018,중구,명동,용산구,한강로동,귀가,37.560008,126.985802,0.014748,37.528107,126.969193,0.035966
1,140.0,162.0,NaN,새벽,평일,2018,서초구,반포1동,강서구,등촌제3동,귀가,37.505081,127.013377,0.025657,37.559270,126.848268,0.173774
2,139.0,160.0,NaN,새벽,평일,2018,강북구,인수동,노원구,하계2동,귀가,37.641473,127.010640,0.036339,37.632001,127.067959,0.058096
3,207.0,218.0,NaN,새벽,평일,2018,관악구,청림동,성북구,안암동,귀가,37.492266,126.958473,0.022919,37.586123,127.021650,0.113139
4,210.0,232.0,NaN,새벽,평일,2018,노원구,월계1동,노원구,상계5동,귀가,37.619921,127.062983,0.020209,37.662734,127.069523,0.043309


### 서울시 행정동별 연도별 보건업 및 사회복지 서비스 종사자 수

In [44]:
seoul_companies = pd.read_csv('./data/extra_data/서울시_사업체현황.csv')

In [45]:
# 필요없는 열 삭제
seoul_companies.drop(['산업대분류별(1)', '동별(1)', '구분별(2)'], axis=1, inplace=True)

# 열 이름 변경
seoul_companies.columns = ['시점', '산업대', '구별', '동별', '구분별', '수']

# 구별 동별 칼럼 소계 없애기
seoul_companies.drop(seoul_companies[seoul_companies.구별.str.contains('소계')].index, inplace=True)
seoul_companies.drop(seoul_companies[seoul_companies.동별.str.contains('소계')].index, inplace=True)

# 사업체 수 삭제
seoul_companies.drop(seoul_companies[seoul_companies.구분별.str.contains('사업체수')].index, inplace=True)

In [47]:
call_taxi_dongs = set(full_data.출발지동.unique())
companies_dongs = set(seoul_companies.동별.unique())
print(set.difference(call_taxi_dongs, community_dongs))
print(set.difference(companies_dongs, call_taxi_dongs))

set()
{'당산2동', '응암2동', '성산2동', '대림1동', '쌍문2동', '중화2동', '구로4동', '답십리2동', '신길6동', '고척1동', '창2동', '구로5동', '중곡2동', '월곡1동', '제기동', '면목3.8동', '수유3동', '성내3동', '사당4동', '구의2동', '둔촌1동', '묵1동', '등촌1동', '성수1가1동', '신길7동', '천호2동', '도봉2동', '장위2동', '구로3동', '약수동', '갈현1동', '장안1동', '신대방2동', '등촌2동', '왕십리2동', '상봉2동', '중화1동', '대림3동', '둔촌2동', '노량진2동', '답십리1동', '천호3동', '돈암2동', '다산동', '등촌3동', '성내1동', '신길3동', '행당1동', '구의3동', '정릉2동', '남가좌1동', '창4동', '자양4동', '암사3동', '방화2동', '구의1동', '이태원1동', '휘경2동', '상일1동', '장안2동', '오류1동', '성산1동', '자양2동', '창신2동', '오류2동', '불광2동', '수유2동', '가양1동', '화곡6동', '창3동', '원효로2동', '창신1동', '양평1동', '홍은1동', '용신동', '신길4동', '신길5동', '상일2동', '숭인1동', '성수2가1동', '전농1동', '이촌1동', '도봉1동', '방화1동', '성내2동', '숭인2동', '정릉4동', '사당3동', '개봉1동', '홍제3동', '화곡3동', '남가좌2동', '면목4동', '암사2동', '독산1동', '상도4동', '사당2동', '창신3동', '면목7동', '사당5동', '화곡1동', '전농2동', '가양2동', '동화동', '길음1동', '대림2동', '성수2가3동', '상도3동', '항동', '면목2동', '이촌2동', '고덕2동', '상도1동', '시흥3동', '위례동', '창5동', '시흥1동', '월곡2동', '당산1동', '화곡8동', '중곡3동', '방화3동', '장위1동', '창1동'

In [49]:
seoul_companies.drop_duplicates(subset=['시점', '동별'], keep='first', inplace=True, ignore_index=True)

In [70]:
# join
final_data = pd.merge(final_data, seoul_companies[['동별', '수', '시점']], how = "left", left_on = ["출발지동", "year"], right_on = ["동별","시점"])
final_data.rename(columns = {"수":"보건업 및 사회복지 서비스업 종사자 수"}, inplace = True)
final_data = final_data.drop(columns = ['동별', '시점'])

In [71]:
print(final_data.shape)
final_data.head()

(5817767, 18)


,dispatch_waiting_time,total_waiting_time,cancel_time,using_time,using_day,year,출발지구,출발지동,목적지구,목적지동,이용목적,출발동_위도,출발동_경도,출발동_최근접_차고지_거리,목적동_위도,목적동_경도,이동거리,보건업 및 사회복지 서비스업 종사자 수
0,136.0,165.0,NaN,새벽,평일,2018,중구,명동,용산구,한강로동,귀가,37.560008,126.985802,0.014748,37.528107,126.969193,0.035966,2156
1,140.0,162.0,NaN,새벽,평일,2018,서초구,반포1동,강서구,등촌제3동,귀가,37.505081,127.013377,0.025657,37.559270,126.848268,0.173774,772
2,139.0,160.0,NaN,새벽,평일,2018,강북구,인수동,노원구,하계2동,귀가,37.641473,127.010640,0.036339,37.632001,127.067959,0.058096,1461
3,207.0,218.0,NaN,새벽,평일,2018,관악구,청림동,성북구,안암동,귀가,37.492266,126.958473,0.022919,37.586123,127.021650,0.113139,324
4,210.0,232.0,NaN,새벽,평일,2018,노원구,월계1동,노원구,상계5동,귀가,37.619921,127.062983,0.020209,37.662734,127.069523,0.043309,759


### 서울시 행정동별 장애인 시설 수

In [52]:
disabled_center = pd.read_csv('./data/extra_data/서울시_장애인시설.csv', encoding='cp949')

In [53]:
# 필요없는 열 삭제
disabled_center.drop(['시설명', '시설코드', '자치구(시)구분', '시설장명', '시군구코드', '시설주소', '전화번호','우편번호' ], axis=1, inplace=True)

# 열 이름 변경
disabled_center.columns = ['시설종류명', '시설종류상세명', '구별', '시설정원','현인원']

# 서울특별시 삭제
disabled_center.drop(disabled_center[disabled_center['구별'].str.contains('서울특별시')].index, inplace=True)

In [54]:
# 구별 갯수 세기
grouped_dis_center = disabled_center['구별'].groupby(disabled_center['구별'])
grouped_dis_center = pd.DataFrame(grouped_dis_center.count())
grouped_dis_center.columns = ['구별_장애인_시설_수']
grouped_dis_center = grouped_dis_center.reset_index()
grouped_dis_center

,구별,구별_장애인_시설_수
0,강남구,55
1,강동구,50
2,강북구,30
3,강서구,58
4,관악구,24
5,광진구,14
6,구로구,25
7,금천구,22
8,노원구,58
9,도봉구,23


In [55]:
call_taxi_gus = set(full_data.출발지구.unique())
community_gus = set(grouped_dis_center.구별.unique())
print(set.difference(call_taxi_gus, community_gus))
print(set.difference(community_gus, call_taxi_gus))

set()
set()


In [72]:
# join
final_data = pd.merge(final_data, grouped_dis_center, how = "left", left_on = "출발지구", right_on = "구별")
final_data = final_data.drop(columns = ['구별'])

In [73]:
print(final_data.shape)
final_data.head()

(5817767, 19)


,dispatch_waiting_time,total_waiting_time,cancel_time,using_time,using_day,year,출발지구,출발지동,목적지구,목적지동,이용목적,출발동_위도,출발동_경도,출발동_최근접_차고지_거리,목적동_위도,목적동_경도,이동거리,보건업 및 사회복지 서비스업 종사자 수,구별_장애인_시설_수
0,136.0,165.0,NaN,새벽,평일,2018,중구,명동,용산구,한강로동,귀가,37.560008,126.985802,0.014748,37.528107,126.969193,0.035966,2156,11
1,140.0,162.0,NaN,새벽,평일,2018,서초구,반포1동,강서구,등촌제3동,귀가,37.505081,127.013377,0.025657,37.559270,126.848268,0.173774,772,30
2,139.0,160.0,NaN,새벽,평일,2018,강북구,인수동,노원구,하계2동,귀가,37.641473,127.010640,0.036339,37.632001,127.067959,0.058096,1461,30
3,207.0,218.0,NaN,새벽,평일,2018,관악구,청림동,성북구,안암동,귀가,37.492266,126.958473,0.022919,37.586123,127.021650,0.113139,324,24
4,210.0,232.0,NaN,새벽,평일,2018,노원구,월계1동,노원구,상계5동,귀가,37.619921,127.062983,0.020209,37.662734,127.069523,0.043309,759,58


### 행정동별 연도별 장애인 수

In [74]:
seoul_disabled = pd.read_csv('./data/extra_data/서울시_장애인구.csv')

In [75]:
# '-' -> NaN 바꾸기
seoul_disabled.장애인구수 = seoul_disabled.장애인구수.str.replace('-','')

# 장애인구 수 형 변환
seoul_disabled.장애인구수 = pd.to_numeric(seoul_disabled.장애인구수)

# '구'로 구분된 데이터 삭제
seoul_disabled.drop(seoul_disabled[seoul_disabled['동별'].str.contains('구')].index, inplace=True)

# 기타로 분류된 데이터 삭제
seoul_disabled.drop(seoul_disabled[seoul_disabled['동별'].str.contains('기타')].index, inplace=True)

# 시점 열에서 '년' 없애기
seoul_disabled.시점 = seoul_disabled.시점.str.replace(' 년', '')

In [76]:
seoul_disabled.head()

,동별,장애유형별,시점,장애인구수
80,사직동,합계,2018,309.0
81,사직동,합계,2019,295.0
82,사직동,합계,2020,284.0
83,사직동,합계,2021,277.0
84,사직동,합계,2022,267.0


In [77]:
seoul_disabled.시점 = pd.to_numeric(seoul_disabled.시점)

In [78]:
call_taxi_dongs = set(full_data.출발지동.unique())
disabled_dongs = set(seoul_disabled.동별.unique())
print(set.difference(call_taxi_dongs, community_dongs))
print(set.difference(disabled_dongs, call_taxi_dongs))

set()
{'당산2동', '응암2동', '성산2동', '대림1동', '쌍문2동', '중화2동', '답십리2동', '신길6동', '고척1동', '창2동', '중곡2동', '월곡1동', '제기동', '면목3.8동', '수유3동', '성내3동', '사당4동', '둔촌1동', '묵1동', '등촌1동', '성수1가1동', '신길7동', '천호2동', '도봉2동', '장위2동', '약수동', '갈현1동', '장안1동', '신대방2동', '등촌2동', '왕십리2동', '상봉2동', '중화1동', '대림3동', '노량진2동', '답십리1동', '천호3동', '돈암2동', '다산동', '등촌3동', '성내1동', '신길3동', '행당1동', '정릉2동', '남가좌1동', '창4동', '자양4동', '암사3동', '방화2동', '이태원1동', '휘경2동', '상일1동', '장안2동', '오류1동', '성산1동', '자양2동', '창신2동', '오류2동', '불광2동', '수유2동', '가양1동', '화곡6동', '창3동', '원효로2동', '창신1동', '양평1동', '홍은1동', '용신동', '신길4동', '신길5동', '상일2동', '숭인1동', '성수2가1동', '전농1동', '이촌1동', '도봉1동', '방화1동', '성내2동', '숭인2동', '정릉4동', '사당3동', '개봉1동', '홍제3동', '화곡3동', '남가좌2동', '면목4동', '암사2동', '독산1동', '상도4동', '사당2동', '창신3동', '면목7동', '사당5동', '화곡1동', '전농2동', '가양2동', '동화동', '길음1동', '대림2동', '성수2가3동', '상도3동', '항동', '면목2동', '이촌2동', '고덕2동', '상도1동', '시흥3동', '위례동', '창5동', '시흥1동', '월곡2동', '당산1동', '화곡8동', '중곡3동', '방화3동', '장위1동', '창1동', '중곡4동', '방학3동', '휘경1동', '시흥5동', '화곡2동', '이태원2동', '시흥2동

In [79]:
seoul_disabled.drop_duplicates(subset=['시점', '동별'], keep='first', inplace=True, ignore_index=True)

In [80]:
# join
final_data = pd.merge(final_data, seoul_disabled[['동별', '장애인구수', '시점']], how = "left", left_on = ["출발지동", "year"], right_on = ["동별","시점"])
final_data = final_data.drop(columns = ['동별', '시점'])

In [81]:
print(final_data.shape)
final_data.head()

(5817767, 20)


,dispatch_waiting_time,total_waiting_time,cancel_time,using_time,using_day,year,출발지구,출발지동,목적지구,목적지동,이용목적,출발동_위도,출발동_경도,출발동_최근접_차고지_거리,목적동_위도,목적동_경도,이동거리,보건업 및 사회복지 서비스업 종사자 수,구별_장애인_시설_수,장애인구수
0,136.0,165.0,NaN,새벽,평일,2018,중구,명동,용산구,한강로동,귀가,37.560008,126.985802,0.014748,37.528107,126.969193,0.035966,2156,11,126.0
1,140.0,162.0,NaN,새벽,평일,2018,서초구,반포1동,강서구,등촌제3동,귀가,37.505081,127.013377,0.025657,37.559270,126.848268,0.173774,772,30,532.0
2,139.0,160.0,NaN,새벽,평일,2018,강북구,인수동,노원구,하계2동,귀가,37.641473,127.010640,0.036339,37.632001,127.067959,0.058096,1461,30,1794.0
3,207.0,218.0,NaN,새벽,평일,2018,관악구,청림동,성북구,안암동,귀가,37.492266,126.958473,0.022919,37.586123,127.021650,0.113139,324,24,679.0
4,210.0,232.0,NaN,새벽,평일,2018,노원구,월계1동,노원구,상계5동,귀가,37.619921,127.062983,0.020209,37.662734,127.069523,0.043309,759,58,1056.0


### 행정동별 연도별 전체 인구 수

In [82]:
seoul_population = pd.read_csv('./data/extra_data/서울시_주민등록인구.csv')

In [83]:
# 동별 칼럼 소계 제거
seoul_population.drop(seoul_population[seoul_population['동별'].str.contains('소계')].index, inplace=True)

# 2023년 데이터 삭제
seoul_population.drop(seoul_population[seoul_population['시점'].str.contains('2023 1/4')].index, inplace=True)

In [84]:
seoul_population.head()

,구별,동별,시점,인구수
7,종로구,사직동,2018,9818
8,종로구,사직동,2019,9815
9,종로구,사직동,2020,9806
10,종로구,사직동,2021,9636
11,종로구,사직동,2022,9355


In [85]:
call_taxi_dongs = set(full_data.출발지동.unique())
population_dongs = set(seoul_population.동별.unique())
print(set.difference(call_taxi_dongs, community_dongs))
print(set.difference(population_dongs, call_taxi_dongs))

set()
{'당산2동', '응암2동', '성산2동', '대림1동', '쌍문2동', '중화2동', '구로4동', '답십리2동', '신길6동', '고척1동', '창2동', '구로5동', '중곡2동', '월곡1동', '제기동', '면목3.8동', '수유3동', '성내3동', '사당4동', '구의2동', '둔촌1동', '묵1동', '등촌1동', '성수1가1동', '신길7동', '천호2동', '도봉2동', '장위2동', '구로3동', '약수동', '갈현1동', '장안1동', '신대방2동', '등촌2동', '왕십리2동', '상봉2동', '중화1동', '대림3동', '둔촌2동', '노량진2동', '답십리1동', '천호3동', '돈암2동', '다산동', '등촌3동', '성내1동', '신길3동', '행당1동', '구의3동', '정릉2동', '남가좌1동', '창4동', '자양4동', '암사3동', '방화2동', '구의1동', '이태원1동', '휘경2동', '상일1동', '장안2동', '오류1동', '성산1동', '자양2동', '창신2동', '오류2동', '불광2동', '수유2동', '가양1동', '화곡6동', '창3동', '원효로2동', '창신1동', '양평1동', '홍은1동', '용신동', '신길4동', '신길5동', '상일2동', '숭인1동', '성수2가1동', '전농1동', '이촌1동', '도봉1동', '방화1동', '성내2동', '숭인2동', '정릉4동', '사당3동', '개봉1동', '홍제3동', '화곡3동', '남가좌2동', '면목4동', '암사2동', '독산1동', '상도4동', '사당2동', '창신3동', '면목7동', '사당5동', '화곡1동', '전농2동', '가양2동', '동화동', '길음1동', '대림2동', '성수2가3동', '상도3동', '항동', '면목2동', '이촌2동', '고덕2동', '상도1동', '시흥3동', '위례동', '창5동', '시흥1동', '월곡2동', '당산1동', '화곡8동', '중곡3동', '방화3동', '장위1동', '창1동'

In [86]:
seoul_population.시점 = pd.to_numeric(seoul_population.시점)

In [87]:
seoul_population.drop_duplicates(subset=['시점', '동별'], keep='first', inplace=True, ignore_index=True)

In [88]:
# join
final_data = pd.merge(final_data, seoul_population[['동별', '인구수', '시점']], how = "left", left_on = ["출발지동", "year"], right_on = ["동별","시점"])
final_data = final_data.drop(columns = ['동별', '시점'])

In [89]:
print(final_data.shape)
final_data.head()

(5817767, 21)


,dispatch_waiting_time,total_waiting_time,cancel_time,using_time,using_day,year,출발지구,출발지동,목적지구,목적지동,...,출발동_위도,출발동_경도,출발동_최근접_차고지_거리,목적동_위도,목적동_경도,이동거리,보건업 및 사회복지 서비스업 종사자 수,구별_장애인_시설_수,장애인구수,인구수
0,136.0,165.0,NaN,새벽,평일,2018,중구,명동,용산구,한강로동,...,37.560008,126.985802,0.014748,37.528107,126.969193,0.035966,2156,11,126.0,3470.0
1,140.0,162.0,NaN,새벽,평일,2018,서초구,반포1동,강서구,등촌제3동,...,37.505081,127.013377,0.025657,37.559270,126.848268,0.173774,772,30,532.0,32109.0
2,139.0,160.0,NaN,새벽,평일,2018,강북구,인수동,노원구,하계2동,...,37.641473,127.010640,0.036339,37.632001,127.067959,0.058096,1461,30,1794.0,33046.0
3,207.0,218.0,NaN,새벽,평일,2018,관악구,청림동,성북구,안암동,...,37.492266,126.958473,0.022919,37.586123,127.021650,0.113139,324,24,679.0,16527.0
4,210.0,232.0,NaN,새벽,평일,2018,노원구,월계1동,노원구,상계5동,...,37.619921,127.062983,0.020209,37.662734,127.069523,0.043309,759,58,1056.0,24087.0


In [90]:
final_data.to_csv('./data/final_data/final_data.csv', index = False)